<h1>Assignment_TaskC_Producer1</h1>

In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv
import pygeohash as gh

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'climate'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    with open ('climate_streaming.csv') as csv_file:
        csv_reader= csv.reader(csv_file,delimiter=",")
        next(csv_reader)
        stream_data = []
        for row in csv_reader:
            stream_data.append(row)
    
    while True:
        sender_id = 1
        index = random.randint(0,len(stream_data)-1)  
        
        temp_data = stream_data[index][0]

        geocode=gh.encode(float(stream_data[index][0]),float(stream_data[index][1]),precision=5)

        for element in stream_data[index][1:]:
            temp_data =temp_data+', '+ element
        temp_data = temp_data+' , '+ geocode    
            
        data = str(sender_id)+", "+str(dt.datetime.now().strftime("%H:%M:%S")) + ', ' + temp_data+ ", Climate"

        publish_message(producer, topic, 'parsed', data)
 
        sleep(5)